In [ ]:
import sys
sys.path.insert(0, '/media/david/work_ssd_2_sata/python/phd/wc_cv/src')

import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import json
from tqdm.notebook import tqdm

from utils import grainPreprocess, grainShow, grainMark, grainDraw, grainApprox, grainStats, grainMorphology,grainGenerate

from scipy import stats
import os
import glob

In [ ]:
# json_data.append({'path': paths[i],
#                   'name': name,
#                   'type': types_dict[name],
#                   'legend': [text],
#                   'density_curve_scatter': [x,y],
#                   'gauss_approx_plot': [x_gauss, y_gauss],
#                   'gauss_approx_data': [{'mus': mus, 'sigmas':sigmas, 'amps':amps}],
#                   'angles_series': all_unique_angels,
#                   })

# Read data

In [ ]:
step=5

file_name = 'wc_cv/cv/data/generated_data/bc_left_(1536,1536,1)_preprocess_step_5_angles.json'
data = open(file_name,encoding='utf-8')
data = json.load(data)['data']

# file_name='data/generated_data/o_bc_left_4x_768_360_768_median_generated_step_5_angles.json'
# file_name='data/generated_data/o_bc_left_4x_768_360_512_median_generated_step_5_angles.json'
# file_name='data/generated_data/o_bc_left_4x_768_360_256_median_generated_step_5_angles.json'

# file_name='wc_cv/cv/data/generated_data/o_bc_left_4x_768_360_768_median_generated_5000_step_1_degrees.json'

# data = open(file_name,encoding='utf-8')
# data = json.load(data)


N = 10
M = 7

# save = True
save = False

save_name=file_name

# indices=[3,0, 2]
indices=[2,0,1]

grainShow.angles_plot_base(data,save_name, step, N, M, indices=indices,save=save)

# All angles in one image, base plot

In [ ]:
def angles_approx_2(data_1, data_2, save_name, step, N, M, indices_1, indices_2, save=False, font_size=20,scatter_size=20, axes_num=3, title=None, iter_index = None):
    alloys_indices=range(len(data_1))

    plt.rcParams['font.size'] = '15'
    
    # fig, axes = plt.subplots(1,axes_num,figsize=(N, M))
    fig, axes = plt.subplots(axes_num, 1, figsize=(N, M))

    markers = ['v', 's', 'D', 'd', 'p', '*','P']
    colors_1 = ['orange', 'red','blue','green', 'indigo', 'brown']
    colors_2 = ['olive', 'cyan','pink','purple', 'blue', 'lime']
    
    sub_titles = ['Large grain','Medium grain ','Small grain' ]

    legends=[]
    metrics_list = []
    for k,(i,j) in enumerate(zip(indices_1, indices_2)):
        legends.append([])
        
        y_original=np.array(data_1[i]['density_curve_scatter'][1])
        y_fake=np.array(data_2[j]['density_curve_scatter'][1])
        w_dist = stats.wasserstein_distance(y_fake, y_original)
        

        tv_dist=0
        
        gauss_real=data_1[i]['gauss_approx_data']
        gauss_fake=data_2[j]['gauss_approx_data']
        
        mus_m = (np.array(gauss_fake['mus']) - np.array(gauss_real['mus']) )/np.array(gauss_real['mus']) 
        sigmas_m = (np.array(gauss_fake['sigmas']) - np.array(gauss_real['sigmas']))/np.array(gauss_real['sigmas'])
        amps_m = (np.array(gauss_fake['amps']) - np.array(gauss_real['amps']))/np.array(gauss_real['amps'])
        
        metrics = f'{w_dist*coef:2.3f} {mus_m[0]*100:.2f} {mus_m[1]*100:.2f} {sigmas_m[0]*100:.2f} {sigmas_m[1]*100:.2f} {amps_m[0]*100:.2f} {amps_m[1]*100:.2f}' 
        # print(metrics)
        # metrics = ''
        
        metrics_list.append(metrics)
        
        # legends[k].append(data_1[i]['name']+' '+data_1[i]['type']+ f' real')
        # legends[k].append(data_2[j]['name']+' '+data_2[j]['type']+ f' fake')
        legends[k].append(data_1[i]['name']+ f" real")
        legends[k].append(data_2[j]['name']+ f" fake")

    for j,i in enumerate(indices_1):
        axes[j].plot(data_1[i]['gauss_approx_plot'][0], 
                 data_1[i]['gauss_approx_plot'][1],
                 color=colors_1[i])
    
    for j,i in enumerate(indices_2):
        axes[j].plot(data_2[i]['gauss_approx_plot'][0], 
                 data_2[i]['gauss_approx_plot'][1],
                 color=colors_2[i])

    for j,i in enumerate(indices_1):
        marker = markers[i]
        axes[j].scatter(data_1[i]['density_curve_scatter'][0],data_1[i]['density_curve_scatter'][1],  marker=marker,color=colors_1[i],s=scatter_size)
        if j==0:
            axes[j].set_ylabel('p(x)', fontsize=font_size)
        if iter_index==2:
            axes[j].set_xlabel('Angles in Co binder, deg.', fontsize=font_size-3)

    for j,i in enumerate(indices_2):
        marker = markers[i]
        axes[j].scatter(data_2[i]['density_curve_scatter'][0],data_2[i]['density_curve_scatter'][1],  marker=marker,color=colors_2[i],s=scatter_size)
        if j==0:
            axes[j].set_ylabel('p(x)', fontsize=font_size)
        if iter_index==0:
            axes[j].set_xlabel('Angles in Co binder, deg.', fontsize=font_size-3)
        

    x = [0,60,120,180,240,300,360]
    
    for j in range(len(axes)):
        # step 1
        # axes[j].set_ylim(0,0.006)
        # axes[j].set_xlim(180,300)
        # x = range(180,300,20)

        # step 2
        # axes[j].set_ylim(0,0.01)
        # axes[j].set_xlim(180,300)
        # x = range(180,300,20)

        # step 3
        # axes[j].set_ylim(0,0.015)
        # axes[j].set_xlim(180,300)
        # x = range(180,300,20)

        # step 4
        # axes[j].set_ylim(0,0.02)
        # axes[j].set_xlim(180,300)
        # x = range(180,300,20)

        # step 5
        axes[j].set_ylim(0,0.025)
        axes[j].set_xlim(180,300)
        x = range(180,300,20)

        axes[j].set_xticks(x, x)
        axes[j].legend(legends[j],loc='upper right')
        # axes[j].set_ylim(0,0.07)


        if iter_index==0:
            axes[j].set_title(sub_titles[j],fontsize=font_size)

    # for i in alloys_indices:
    #     print(data_1[i]['legend'])
    # print('-----')
    # for i in alloys_indices:
    #     print(data_2[i]['legend'])
    
    # if title is not None:
    #     if iter_index==0:
    #         fig.suptitle(title,fontsize=font_size, y=1.02,) 
    #     else:
    #         fig.suptitle(title,fontsize=font_size, y=0.95,) 
    # else:
    #     fig.suptitle(save_name,fontsize=font_size) 
    
    if save:
        plt.savefig(save_name, bbox_inches = "tight" )
    
    
    plt.show()
    
    return metrics_list

In [ ]:
step=5

# file_name_1='data/generated_data/o_bc_left_step_5_angles.json'
# file_name_1 = 'wc_cv/cv/data/generated_data/bc_left_(1536,1536,1)_preprocess_step_2_angles.json'
file_name_1 = f'wc_cv/cv/data/generated_data/bc_left_(1536,1536,1)_preprocess_step_{step}_angles.json'

# file_name_2_1='wc_cv/cv/data/generated_data/o_bc_left_4x_768_360_768_median_generated_5000_step_2_degrees.json'
file_name_2_1=f'wc_cv/cv/data/generated_data/o_bc_left_4x_768_360_768_median_generated_5000_step_{step}_degrees.json'

# file_name_2_1='wc_cv/cv/data/generated_data/o_bc_left_4x_768_360_768_median_generated_step_5_angles.json'
# file_name_2_2='wc_cv/cv/data/generated_data/o_bc_left_4x_768_360_512_median_generated_step_5_angles.json'
# file_name_2_3='wc_cv/cv/data/generated_data/o_bc_left_4x_768_360_256_median_generated_step_5_angles.json'

# file_name_2_1='data/generated_data/o_bc_left_4x_768_360_768_median_based_Ultra_Co25_generated_step_5_angles.json'
# file_name_2_2='data/generated_data/o_bc_left_4x_768_360_512_median_based_Ultra_Co25_generated_step_5_angles.json'
# file_name_2_3='data/generated_data/o_bc_left_4x_768_360_256_median_based_Ultra_Co25_generated_step_5_angles.json'

titles = ['768x768', '512x512','256x256']


# names = [file_name_2_1, file_name_2_2, file_name_2_3]
names = [file_name_2_1]

# N = 20*2/3

# N = 20
# M = 5

N = 20
M = 20


step=5

coef=1000
    
save = True
# save = False

data_1 = open(file_name_1,encoding='utf-8')
data_1 = json.load(data_1)['data']

metrics_all=[]

columns = f'w_dist*{coef} mu_1,% mu_2,%  sigma_1,% sigma_2,% amp_1,% amp_2,%'

for i,file_name_2 in enumerate(names):

    save_name=file_name_2.split('/')[-1]

    save_name='wc_cv/cv/'+save_name[:-6]

    indices_1=[3,0, 2]
    indices_2=[2, 0, 1]

    data_2 = open(file_name_2,encoding='utf-8')
    data_2 = json.load(data_2)

    # print(columns)
    metrics = angles_approx_2(data_1,data_2,save_name, step, N, M, indices_1=indices_1,indices_2=indices_2,
                              save=save, axes_num=3, title=titles[i], iter_index=i)
    metrics_all.extend(metrics)
    

print(columns)
for m in metrics_all:
    print(m)

# Developing

In [ ]:
# json_data.append({'path': paths[i],
#                   'name': name,
#                   'type': types_dict[name],
#                   'legend': [text],
#                   'density_curve_scatter': [x,y],
#                   'gauss_approx_plot': [x_gauss, y_gauss],
#                   'gauss_approx_data': [{'mus': mus, 'sigmas':sigmas, 'amps':amps}],
#                   'angles_series': all_unique_angels,
#                   })

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(10, 5))
# fig, axes = plt.subplots(1,3,figsize=(N, M))

markers = ['v', 's', 'D', 'd', 'p', '*','P']
colors_1 = ['orange', 'red','blue','green', 'indigo', 'brown']
# colors_2 = ['olive', 'cyan','pink','purple', 'blue', 'lime']

#     colors_1 = ['orange', 'red','blue','green', 'indigo', 'brown']
#     colors_2 = ['olive', 'pink','pink','purple', 'blue', 'lime']

# sub_titles = ['Large grain','Medium grain ','Small grain' ]

names_dict = {'Ultra_Co11': 'средние зерна',
              'Ultra_Co25': 'мелкие зерна',
              'Ultra_Co8': 'средне-мелкие зерна',
              'Ultra_Co6_2': 'крупные зерна',
              'Ultra_Co15': 'средне-мелкие зерна'}



######
# Real images

# indices_1= [0, 1, 4]
indices_1= [1]

folder_path = 'data/gen_data_5_degrees/o_bc_left_4x_768_256x256_N360'
jsons_names = glob.glob(folder_path+'/*.json')

ethalon_name = 'data/gen_data_5_degrees/o_bc_left_4x_768_256x256_N360.json'
ethalon_dist = json.load(open(ethalon_name,encoding='utf-8'))

# iterate class types for real images
legends=[]
for i, idx in enumerate(indices_1):
    tmp_x=[]
    tmp_y=[]
    for name in jsons_names:
        data = json.load(open(name,encoding='utf-8'))
        y_fake = data[idx]['density_curve_scatter'][1]
        w_dist = stats.wasserstein_distance(y_fake, ethalon_dist[idx]['density_curve_scatter'][1])
        N=int(name.split('/')[-1].split('.')[0][1:])
        tmp_x.append(N)
        tmp_y.append(w_dist)
        class_name = data[idx]['name']
    # legends.append(class_name + ' '+names_dict[class_name])
    legends.append('original images')
    plt.plot(tmp_x, tmp_y, '-o')

######

Ns=np.array([100, 200, 500, 1000, 10000])

plt.plot(Ns, 1/(np.sqrt(Ns)*100), '-o')

legends.extend([f'$10^{-2}/(sqrt(N))$'])

######

folders = ['data/gen_data_5_degrees/o_bc_left_4x_768_360_median_rgb_256x256-gpus4-batch256_step23040_N10000',
           'data/gen_data_5_degrees/o_bc_left_4x_768_360_median_rgb_256x256-gpus8-batch512_step53248_N10000',
           'data/gen_data_5_degrees/o_bc_left_4x_768_360_median_rgb_256x256-gpus8-batch512_step94208_N10000']

gan_legends=['gan_4gpu_step23040_fid173_days6', 
             'gan_8gpu_step53248_fid155_days11', 
             'gan_8gpu_step94208_fid181_days11']

legends.extend(gan_legends)
for i, folder in enumerate(folders):
    jsons_names = glob.glob(folder+'/*.json')
    tmp_x=[]
    tmp_y=[]
    for name in jsons_names:
        data = json.load(open(name,encoding='utf-8'))
        y_fake = data[0]['density_curve_scatter'][1]
        w_dist = stats.wasserstein_distance(y_fake, ethalon_dist[0]['density_curve_scatter'][1])
        N=int(name.split('/')[-1].split('.')[0][1:])
        tmp_x.append(N)
        tmp_y.append(w_dist)
    indices=np.argsort(tmp_x)
    tmp_x = np.array(tmp_x)
    tmp_y = np.array(tmp_y)
    plt.plot(tmp_x[indices], tmp_y[indices], '-o', color=colors_1[i], marker=markers[i])


######

folders = ['data/gen_data_5_degrees/diffusion_o_bc_left_4x_768_360_256_median_N500',]

dif_legends=['diff_100epochs_days0.6']

for i, folder in enumerate(folders):
    jsons_names = glob.glob(folder+'/*.json')
    tmp_x=[]
    tmp_y=[]
    for name in jsons_names:
        data = json.load(open(name,encoding='utf-8'))
        y_fake = data[0]['density_curve_scatter'][1]
        w_dist = stats.wasserstein_distance(y_fake, ethalon_dist[0]['density_curve_scatter'][1])
        N=int(name.split('/')[-1].split('.')[0][1:])
        tmp_x.append(N)
        tmp_y.append(w_dist)
    legends.extend(dif_legends)
    indices=np.argsort(tmp_x)
    tmp_x = np.array(tmp_x)
    tmp_y = np.array(tmp_y)
    # plt.plot(tmp_x[indices], tmp_y[indices], '-o', color=colors_1[i], marker=markers[i])
    plt.plot(tmp_x[indices], tmp_y[indices], '-o')


######


plt.legend(legends)
plt.xscale("log")
plt.ylabel('Wasserstein distance')
plt.xlabel('Number of images')
plt.title('Ultra_Co11 средние зерна 256x256')
plt.show()

In [ ]:
legends

In [ ]:
os.listdir(folder)

In [ ]:
step=5


file_name_1 = f'wc_cv/cv/data/generated_data/bc_left_(1536,1536,1)_preprocess_step_{step}_angles.json'

# file_name_2_1='wc_cv/cv/data/generated_data/o_bc_left_4x_768_360_768_median_generated_step_5_angles.json'
# file_name_2_2='wc_cv/cv/data/generated_data/o_bc_left_4x_768_360_512_median_generated_step_5_angles.json'
# file_name_2_3='wc_cv/cv/data/generated_data/o_bc_left_4x_768_360_256_median_generated_step_5_angles.json'



titles = ['768x768', '512x512','256x256']


# names = [file_name_2_1, file_name_2_2, file_name_2_3]
names = [file_name_2_1]

# N = 20*2/3

# N = 20
# M = 5

N = 20
M = 20


step=5

coef=1000
    
save = True
# save = False

data_1 = open(file_name_1,encoding='utf-8')
data_1 = json.load(data_1)['data']

metrics_all=[]

columns = f'w_dist*{coef} mu_1,% mu_2,%  sigma_1,% sigma_2,% amp_1,% amp_2,%'

for i,file_name_2 in enumerate(names):

    save_name=file_name_2.split('/')[-1]

    save_name='wc_cv/cv/'+save_name[:-6]

    indices_1=[3,0, 2]
    indices_2=[2, 0, 1]

    data_2 = open(file_name_2,encoding='utf-8')
    data_2 = json.load(data_2)

    # print(columns)
    metrics = angles_approx_2(data_1,data_2,save_name, step, N, M, indices_1=indices_1,indices_2=indices_2,
                              save=save, axes_num=3, title=titles[i], iter_index=i)
    metrics_all.extend(metrics)
    

print(columns)
for m in metrics_all:
    print(m)